# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning', 'R. E. Hviding']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
R. E. Hviding  ->  R. E. Hviding  |  ['R. E. Hviding']
F. Walter  ->  F. Walter  |  ['F. Walter']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
T. Henning  ->  T. Henning  |  ['T. Henning']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']


Y. Wang  ->  Y. Wang  |  ['Y. Wang']
J. Müller  ->  J. Müller-Horn  |  ['J. Müller']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
J. Müller-Horn  ->  J. Müller-Horn  |  ['J. Müller-Horn']
Arxiv has 84 new papers today
          9 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/9 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2601.20929


extracting tarball to tmp_2601.20929...

 done.


Found 211 bibliographic references in tmp_2601.20929/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2601.20948


...

/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure metal_robust
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure metal_robust as tmp_2601.21292/./metal_robust.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


Found 195 bibliographic references in tmp_2601.21292/main.bbl.
Error retrieving bib data for choiMesaIsochronesStellar2016: 'author'
Retrieving document from  https://arxiv.org/e-print/2601.21693
extracting tarball to tmp_2601.21693... done.
Retrieving document from  https://arxiv.org/e-print/2601.21745


extracting tarball to tmp_2601.21745... done.
Retrieving document from  https://arxiv.org/e-print/2601.22071
extracting tarball to tmp_2601.22071... done.


K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
J. Müller-Horn  ->  J. Müller-Horn  |  ['J. Müller-Horn']


Issues with the citations
repeated bibliography entry: Prvsa_Zwitter_2005


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.20929-b31b1b.svg)](https://arxiv.org/abs/2601.20929) | **Little Red Dot $-$ Host Galaxy $=$ Black Hole Star: A Gas-Enshrouded Heart at the Center of Every Little Red Dot**  |
|| W. Q. Sun, et al. -- incl., <mark>A. d. Graaff</mark>, <mark>R. E. Hviding</mark>, <mark>F. Walter</mark> |
|*Appeared on*| *2026-01-30*|
|*Comments*| *Submitted to the Open Journal of Astrophysics. Main results in Figs. 5 (BH* stack), 12 (explanation of the LRD SED), and 15 (hidden BH*s in blue galaxies). Comments warmly welcomed!*|
|**Abstract**|            The central engines of Little Red Dots (LRDs) may be ``black hole stars" (BH*s), early stages of black hole growth characterized by dense gas envelopes. So far, the most direct evidence for BH*s comes from a handful of sources where the host galaxy is completely outshone as suggested by their remarkably steep Balmer breaks. Here we present a novel scheme to disentangle BH*s from their host galaxies assuming that the [OIII]5008Å line arises exclusively from the host. Using a sample of 98 LRDs ($z$~$2-9$) with high quality NIRSpec/PRISM spectra, we demonstrate that the host-subtracted median stack displays a Balmer break $>2\times$ stronger than massive quiescent galaxies, with the rest-optical continuum resembling a blackbody-like SED ($T_{\rm{eff}}$~$4050$ K, $\log(L_{\rm{bol}})$~$43.9$ erg s$^{-1}$, $R_{\rm{eff}}$~$1300$ au). We measure a steep Balmer decrement (H$\alpha$/H$\beta>10$) and numerous density-sensitive features (e.g., FeII, HeI, OI). These are hallmark signatures of dense gas envelopes, providing population-level evidence that BH*s indeed power LRDs. In the median LRD, BH*s account for $\sim20\%$ of the UV emission, $\sim50\%$ at the Balmer break, and $\sim90\%$ at wavelengths longer than H$\alpha$ with the remainder arising from the host. BH*s preferentially reside in low-mass galaxies ($M_{\rm{\star}}$~$10^{8}\,{\rm M}_{\rm{\odot}}$) undergoing recent starbursts, as evidenced by extreme emission line EWs (e.g., [OIII]5008Å~$1100$Å, CIII]~$12$Å), thereby favoring BH* origins linked to star-formation. We show V-shaped LRD selections are biased to high BH*/host fractions ($\gtrsim60\%$ at 5500Å) -- less dominant BH*s may be powering JWST's blue broad-line AGN. We find BH*s are so commonplace and transient (duty cycle $\sim1\%$, lifetime $\sim10$ Myrs) that every massive black hole may have once shone as a BH*.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.20948-b31b1b.svg)](https://arxiv.org/abs/2601.20948) | **Euclid: Early Release Observations -- The star-formation history of massive early-type galaxies in the Perseus cluster**  |
|| S. Martocchia, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2026-01-30*|
|*Comments*| *21 pages, 8 figures, accepted for publication in A&A*|
|**Abstract**|            The Euclid Early Release Observations (ERO) programme targeted the Perseus galaxy cluster in its central region over 0.7deg$^2$. We combined the exceptional image quality and depth of the ERO-Perseus with FUV and NUV observations from GALEX and AstroSat/UVIT, as well as $ugrizH\alpha$ data from MegaCam at the CFHT, to deliver FUV-to-NIR magnitudes of the 87 brightest galaxies within the Perseus cluster. We reconstructed the star-formation history (SFH) of 59 early-type galaxies (ETGs) within the sample, through the spectral energy distribution (SED) fitting code CIGALE and state-of-the-art stellar population (SP) models to reproduce the galactic UV emission from hot, old, low-mass stars (i.e. the UV upturn). In addition, for the six most massive ETGs in Perseus [stellar masses $\log_{10}(M_{\ast}/M_{\odot}) \geq 10.3$], we analysed their spatially resolved SP through a radial SED fitting. In agreement with our previous work on Virgo ETGs, we found that (i) the majority of ETGs needs the presence of an UV upturn to explain their FUV emission, with temperatures $\langle T_{\rm UV}\rangle$~33800 K; (ii) ETGs have grown their stellar masses quickly, with SF timescales $\tau\lesssim 1500$ Myr. We found that all ETGs in the sample have formed more than about 30% of their stellar masses at z~5, up to ~100%. At z~5, the stellar masses of the most massive nearby ETGs, which have present-day stellar masses $\log_{10}(M_{\ast}/M_{\odot})\gtrsim 10.8$, are then found to be comparable to those of the red quiescent galaxies observed by JWST at similar redshifts (z>4.6). This study can be extended to ETGs in the 14000 deg$^2$ extragalactic sky that will soon be observed by Euclid, in combination with those from other major upcoming surveys (e.g. Rubin/LSST), and UV observations, to ultimately assess whether the nearby massive ETGs represent the progeny of the massive high-z JWST red quiescent galaxies.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.20966-b31b1b.svg)](https://arxiv.org/abs/2601.20966) | **The Third Option: Color Phase Curves to Characterize the Atmospheres of Temperate Rocky Exoplanets**  |
|| D. Deming, et al. -- incl., <mark>L. Kreidberg</mark> |
|*Appeared on*| *2026-01-30*|
|*Comments*| *14 pages, 6 figures, submission to The Open Journal of Astrophysics*|
|**Abstract**|            Detecting and characterizing the atmospheres of rocky exoplanets has proven to be challenging for JWST. Transit spectroscopy of the TRAPPIST-1 planets has been impacted by the effects of spots and faculae on the host star. Secondary eclipses have detected hot rocks, but evidence for atmospheres has been difficult to obtain. However, there is a third option that we call color phase curves. This method will apply to synchronously rotating non-transiting planets as well as transiting planets. A color phase curve uses photometry at a long-IR wavelength near the peak of the planetary thermal emission (e.g., 21 microns) divided by photometry at a shorter wavelength where the star dominates more strongly (e.g., 12 microns). We avoid wavelengths having potentially strong molecular absorption (e.g., 15 microns) to minimize degeneracies in the color phase curve, and we aim to detect and characterize the planetary atmosphere via its longitudinal heat transfer. The ratio of two wavelengths observed nearly simultaneously is designed to isolate thermal emission from the planet, discriminate against the star, and largely cancel instrumental systematic effects. Moreover, we show that invoking mass-radius relations, and using self-consistent physical models, will permit the longitudinal heat transfer to be measured independent of the orbital inclination. Radial velocity surveys are detecting many new exoplanets, including temperate rocky worlds with Earth-like masses. Most of those planets will not transit, but color phase curves have the potential to detect and characterize their atmospheres.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.21072-b31b1b.svg)](https://arxiv.org/abs/2601.21072) | **Thermal emission spectra of the ultra-hot Jupiter WASP-33 b**  |
|| Q. Zou, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2026-01-30*|
|*Comments*| *Accepted for publication in Astronomy and Astrophysics(12 pages, 16 figures, 5 tables)*|
|**Abstract**|            Observations of exoplanetary atmospheres provide critical insights into their chemical composition, formation and evolution history. Ultra-hot Jupiters serve as excellent targets for atmospheric characterization; studies of these planets may yield key understanding of gas giant's formation and evolution history. We present a thermal emission study of WASP-33 b's dayside atmosphere, based on two secondary eclipse observations with CFHT/WIRCam in two specific narrow band filters, namely the CO and CH4$_{\rm on}$ filters, and archival data with HST/WFC3 and Spitzer. Stellar pulsations of the host star induce some quasi-periodic photometric variations, particularly in the CH4$_{\rm on}$ band, which are modelled and corrected in the high-precision differential light curves. An eclipse depth of $1565.2^{+228.6}_{-237.5}$ ppm and $914.3^{+56.1}_{-57.0}$ ppm is determined for the CO and CH4$_{\rm on}$ bands, respectively. Combined with HST/WFC3 and Spitzer data, our joint retrieval of WASP-33 b's dayside atmosphere reveals a high metallicity ([Fe/H] $= 1.52^{+0.35}_{-0.52}$), high C/O ratio (C/O $= 0.78^{+0.03}_{-0.04}$), and a thermal inversion layer, suggesting a formation history involving metal-rich gas accretion. We confirm the presence of the molecules H$_{2}$O, H$^{-}$ and CO, and report a tentative detection of TiO in the dayside atmosphere of WASP-33 b. Future higher precision observations with JWST may provide better understand constraints on the chemical abundances of oxygen and refractory element abundances to better WASP-33 b's formation and evolutionary pathway.         |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.21292-b31b1b.svg)](https://arxiv.org/abs/2601.21292) | **Evaluating Classifications of Extremely Metal-poor Candidates Selected from Gaia XP Spectra**  |
|| R. Thai, et al. -- incl., <mark>H.-W. Rix</mark> |
|*Appeared on*| *2026-01-30*|
|*Comments*| *26 pages, 12 figures. Submitted to AJ*|
|**Abstract**|            Extremely metal-poor stars are intrinsically rare, but emerging methods exist to accurately classify them from all-sky Gaia XP low-resolution spectra. To assess their overall accuracy for targeting metal-poor stars, we present a high-resolution spectroscopic followup of 75 very metal-poor candidates selected from the catalog by R. Andrae, V. Chandra, and H. W. Rix. We discover 2 new extremely metal-poor ($\rm{[Fe / H]}<-3$) stars and 20 new very metal-poor ($\rm{[Fe/H]} < -2$) stars. Abundances of up to 22 elements are derived from 1D local thermodynamic equilibrium analysis and kinematic parameters are derived using Gaia astrometry and spectroscopic radial velocities. The chemodynamical properties are mostly consistent with expectations for halo stars, but we discover an Mg-enhanced CEMP star ($\mathrm{[Mg/Fe]} = 0.89$) and an Mg-poor star from an accreted ultra-faint dwarf galaxy. The Gaia XP metallicity estimates are consistent with our $\rm{[Fe/H]}$ measurements down to $\rm{[Fe/H]}\sim -3.0$, but estimates worsen in highly extincted regions. We find that 4 other XP-based metallicity catalogs succeed in mitigating contaminants and can also classify metal-poor stars robustly to $\rm{[Fe/H]}\sim -3.0$. Our results demonstrate the utility of Gaia XP spectra for identifying the most metal-poor stars across the Galaxy.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.22071-b31b1b.svg)](https://arxiv.org/abs/2601.22071) | **BE Lyncis is not a Black Hole Binary: Lessons From Gaia and Hipparcos Astrometry**  |
|| P. Nagarajan, et al. -- incl., <mark>K. El-Badry</mark>, <mark>J. Müller-Horn</mark> |
|*Appeared on*| *2026-01-30*|
|*Comments*| *6 pages, 1 figure*|
|**Abstract**|            BE Lyncis (BE Lyn) is a well-studied high-amplitude $\delta$ Scuti variable star (HADS). Recently, Niu et al. (2026) analyzed a 39-year baseline of times of maximum light of BE Lyn, reporting that it is the most eccentric binary known ($e \approx 0.9989$) and hosts the nearest black hole (BH) known to date. We analyze Hipparcos and Gaia astrometry of BE Lyn, predicting what the observed proper motion anomaly (PMA) over the 25 year baseline between the two missions would be were the companion really a $\gtrsim 17.5\,M_{\odot}$ BH. We find that the predicted PMA is at least an order of magnitude larger than the observed value of $\approx 1.7 \pm 0.8$ mas yr$^{-1}$, regardless of the assumed orientation of the orbit. We predict the expected Gaia DR3 RUWE for different orientations of the putative BH binary, finding that it ranges from $\approx 2.5$-$4.0$, much larger than the reported value of $1.073$. The observed value is instead consistent with a low-mass secondary or a single star. We find that BE Lyncis would have received a 7-parameter acceleration solution if it were a BH binary, in contradiction with its absence from the Gaia DR3 non-single star catalogs. Finally, we show that the reported orbit is impossible because the luminous star would overflow its Roche lobe at periastron, irrespective of inclination. We recommend caution in interpreting light-travel time effect (LTTE) models that require very high eccentricities, face-on inclinations, or large companion masses. The observed pulsation timing variations are most likely simply a result of red noise or pulsation phase evolution.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.21693-b31b1b.svg)](https://arxiv.org/abs/2601.21693) | **Pulse-resolved Classification and Characteristics of Long-duration GRBs with \emph{Swift}-BAT Data.I. Precursors versus Main Bursts**  |
|| L. Li, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2026-01-30*|
|*Comments*| *33 pages, 6 figures (including 73 panels), 6 tables*|
|**Abstract**|            We present a systematic pulse-by-pulse analysis of 22 long-duration GRBs observed by \emph{Swift}, each exhibiting a well-separated precursor before the main burst. We compare duration, spectral hardness ratio, minimum variability timescale (MVT), and spectral lag between these components. Both precursors and main bursts have durations and hardness broadly consistent with Type II GRBs. However, precursors show longer MVTs (by factors of 3-10) and diverse lags with near-zero median values, while main bursts display variable MVTs and positive lags. These differences suggest precursors may originate from distinct dissipation conditions, possibly due to cocoon shock breakout or early magnetically dominated outflows. Despite temporal differences, both episodes are consistent with a single collapsar origin, providing no evidence for dual-progenitor events. Our findings support pulse-resolved classification and show that precursors offer critical insights into jet formation and pre-burst activity.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.21745-b31b1b.svg)](https://arxiv.org/abs/2601.21745) | **Asymptotic power spectra and visibilities of damped mixed modes**  |
|| <mark>J. Müller</mark>, et al. |
|*Appeared on*| *2026-01-30*|
|*Comments*| *accepted by A&A, 25 pages, 22 figures*|
|**Abstract**|            Recent observational studies of red giant stars have estimated the visibility of their mixed oscillation modes, which is a proxy of the average energy of these modes. Among other things, they demonstrated that although the damping rate of the oscillations in the core of many red giants appears to be negligible, other red giants exhibit high core damping rates that are sometimes consistent with the infinite value limit. Up until now, it has not been possible to link the mixed mode visibilities to core damping rates in a quantitative way. In this study, we use the progressive wave picture to derive an analytical function expressing the approximate resonance pattern of red giants up to a proportionality factor. This function can model the influence of the damping on the oscillations, as well as take into account other effects such as mode asymmetries. In particular, this expression can be used to obtain a quantitative estimate for the visibility of mixed modes and to predict the detectability of mixed mode and multiplet signatures under different core damping rates. Here, we conduct a parameter study to investigate how the damping processes affect these aspects. We find that the visibility approaches the value expected for an infinite core damping rate already at finite values. Furthermore, we find that both the mixed mode and the multiplet signatures disappear at finite core damping rates. This implies that the observational characteristics of red giants with finite core damping rates can appear as if their core damping rate were infinite, providing an explanation for the observed populations. Moreover, we have used our method to quantitatively estimate the core damping rates of red giants with unusually low mixed mode amplitudes from their observed visibilities.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2601.20929/./figs/fig3_main_stack.png', 'tmp_2601.20929/./figs/fig6_bhstar_frac_vs_lam.png', 'tmp_2601.20929/./figs/context_v2.png']
copying  tmp_2601.20929/./figs/fig3_main_stack.png to _build/html/
copying  tmp_2601.20929/./figs/fig6_bhstar_frac_vs_lam.png to _build/html/
copying  tmp_2601.20929/./figs/context_v2.png to _build/html/
exported in  _build/html/2601.20929.md
    + _build/html/tmp_2601.20929/./figs/fig3_main_stack.png
    + _build/html/tmp_2601.20929/./figs/fig6_bhstar_frac_vs_lam.png
    + _build/html/tmp_2601.20929/./figs/context_v2.png
found figures ['tmp_2601.20948/./figs/FUV_NUV_Mass_integr_virgo_perseus.png', 'tmp_2601.20948/./figs/FUV_VIS_Mass_integr_virgo_perseus.png', 'tmp_2601.20948/./figs/fuv_nuv_only_SMA.png', 'tmp_2601.20948/./figs/fuv_VIS_only_SMA.png', 'tmp_2601.20948/./figs/sed_integrated_results_mask.png']
copying  tmp_2601.20948/./figs/FUV_NUV_Mass_integr_virgo_perseus.png to _build/html/
copying  tmp_2601.20948/./figs/FUV_VIS_Mass_in

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\xiion}{\xi_{\rm{ion}}}$
$\newcommand{\halpha}{H\ensuremath{\alpha}}$
$\newcommand{\hbeta}{H\ensuremath{\beta}}$
$\newcommand{\mstar}{\ensuremath{\log(M_{\rm{\star}}/ {\rm M}_{\rm{\odot}})}}$
$\newcommand{\orcidauthor}[3]{\author{\href{http://orcid.org/#1}{#2^{#3}}}}$
$\newcommand{\nion}[2]{#1 \textsc{#2}}$</div>



<div id="title">

# $\vspace{-1cm}$ Little Red Dot $-$ Host Galaxy $=$ Black Hole Star:\ A Gas-Enshrouded Heart at the Center of Every Little Red Dot $\vspace{-1.75cm}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2601.20929-b31b1b.svg)](https://arxiv.org/abs/2601.20929)<mark>Appeared on: 2026-01-30</mark> -  _Submitted to the Open Journal of Astrophysics. Main results in Figs. 5 (BH* stack), 12 (explanation of the LRD SED), and 15 (hidden BH*s in blue galaxies). Comments warmly welcomed!_

</div>
<div id="authors">

W. Q. Sun, et al. -- incl., <mark>A. d. Graaff</mark>, <mark>R. E. Hviding</mark>, <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** The central engines of Little Red Dots (LRDs) may be "black hole stars" (BH*s), early stages of black hole growth characterized by dense gas envelopes. So far, the most direct evidence for BH*s comes from a handful of sources where the host galaxy is completely outshone as suggested by their remarkably steep Balmer breaks. Here we present a novel scheme to disentangle BH*s from their host galaxies assuming that the [ $\ion{O}{3}$ ] 5008Å line arises exclusively from the host. Using a sample of 98 LRDs ( $z\approx2-9$ ) with high quality NIRSpec/PRISM spectra, we demonstrate that the host-subtracted median stack displays a Balmer break $>2\times$ stronger than massive quiescent galaxies, with the rest-optical continuum resembling a blackbody-like SED ( $T_{\rm{eff}}\approx4050$ K, $\log(L_{\rm{bol}})\approx43.9$ erg s $^{-1}$ , $R_{\rm{eff}}\approx1300$ au). We measure a steep Balmer decrement (H $\alpha$ /H $\beta>10$ ) and numerous density-sensitive features (e.g., $\ion{Fe}{2}$ , $\ion{He}{1}$ , $\ion{O}{1}$ ). These are hallmark signatures of dense gas envelopes, providing population-level evidence that BH*s indeed power LRDs. In the median LRD, BH*s account for $\sim20\%$ of the UV emission, $\sim50\%$ at the Balmer break, and $\sim90\%$ at wavelengths longer than H $\alpha$ with the remainder arising from the host. BH*s preferentially reside in low-mass galaxies ( $M_{\rm{\star}}\approx 10^{8} {\rm M}_{\rm{\odot}}$ ) undergoing recent starbursts, as evidenced by extreme emission line EWs (e.g., [ $\ion{O}{3}$ ] 5008Å $\approx1100$ Å, $\ion{C}{3}$ ] $\approx12$ Å), thereby favoring BH* origins linked to star-formation. We show V-shaped LRD selections are biased to high BH*/host fractions ( $\gtrsim60\%$ at 5500Å) -- less dominant BH*s may be powering JWST's blue broad-line AGN. We find BH*s are so commonplace and transient (duty cycle $\sim1\%$ , lifetime $\sim10$ Myrs) that every massive black hole may have once shone as a BH*.

</div>

<div id="div_fig1">

<img src="tmp_2601.20929/./figs/fig3_main_stack.png" alt="Fig6" width="100%"/>

**Figure 6. -** **Stacks of $\text{LRD** - \text{Host}$  difference spectra (top) and host galaxies (bottom-right) constructed by applying our decomposition procedure to each of the 98 LRDs in our sample (stacked in the bottom-left).}**Top:** The central engine of the _median_ LRD shows a steep Balmer break ($f^{\nu}(4050\rm{Å})$/$f^{\nu}(3670\rm{Å})\approx7$), extremely strong H$\alpha$ emission ($\approx850$Å), a steep Balmer decrement (H$\alpha$/H$\beta\approx11$), and strong $\ion${O}{1} and Fe emission. These are hallmark signatures of radiative transfer in a dense gas envelope, strongly supporting the picture that the central engine of the typical LRD is indeed a BH*. Note that the [$\ion${O}{3}] line by construction has zero luminosity, but appears over/under-subtracted because the kinematic width of the line is never exactly the same across the LRDs and the candidate hosts. **Bottom-Left:** In LRDs, these hallmark BH* features are mixed with galaxy light and are challenging to discern (e.g., there is only a weak Balmer break), underscoring the need for our procedure. **Bottom-Right:** The median host galaxy is highly star-forming, as evidenced by strong rest-optical emission lines (EW($\ion${O}{3}+H$\beta$)$\approx1700$Å) that are $\approx2-3\times$ stronger than the typical galaxy at these redshifts \citep[e.g.,][]{Endsley24}. The detection of strong $\ion${C}{3}]($\approx12$Å, $>2\times$ the typical EW at $z\approx5$) points to the hosts having undergone a recent burst \citep[e.g.,][]{Roberts-Borsani25}. (*fig:main_stack*)

</div>
<div id="div_fig2">

<img src="tmp_2601.20929/./figs/fig6_bhstar_frac_vs_lam.png" alt="Fig12" width="100%"/>

**Figure 12. -** **Wavelength-dependent BH*/LRD fractions of the BH* stack (left) and three substacks divided by $L_{5500**$ (right).} The faint substack has $38.5 < \log\left(L_{\rm{5500}} / \text{erg} \text{s}^{-1}\right) < 39.5$; the intermediate substack has $39.5 < \log\left(L_{\rm{5500}} / \text{erg} \text{s}^{-1}\right) < 40.5$; the bright substack has $40.5 < \log\left(L_{\rm{5500}} / \text{erg} \text{s}^{-1}\right) < 41.5$. Generally, the BH* provides relatively modest contributions in the rest-UV, but there is a sharp increase around $\approx3600-4000$Å (not necessarily at the Balmer limit, but redwards for the faintest). At redder wavelengths the BH* begins to dominate, accounting for almost all the light around $\approx1\mu$m. There is a strong luminosity dependence to the BH* contribution -- the brightest BH*s (green curve) are also luminous in the UV, contributing $\approx50\%$ around the region where the Fe UV forest is observed (corresponding to the prominent peaks in the UV). On the other hand, fainter BH*s (blue) make their presence known only in the rest-optical and are effectively invisible at bluer wavelengths. The sharp, almost vertical transition in BH* fraction redwards of the Balmer limit is luminosity dependent, and occurs only in the most luminous sources, whereas in fainter BH*s we witness a more gradual transition. Emission lines show expected trends -- forbidden lines such as [$\ion${O}{3}](by construction) and the [$\ion${S}{3}] doublet are prominent in hosts relative to the BH*.
     (*fig:bhstarfrac*)

</div>
<div id="div_fig3">

<img src="tmp_2601.20929/./figs/context_v2.png" alt="Fig7" width="100%"/>

**Figure 7. -** **The $\text{LRD**-\text{Host}$ difference stack has unusual properties relative to typical AGN and galaxies.}**Left:** We compare the exceptional H$\alpha$ EW and Balmer decrement of the $\text{LRD}-\text{Host}$ stack against a compilation of $z\approx0-0.6$ SDSS quasars \citep[][]{Wu22SDSS}. Our stack has few peers among these local quasars, hinting that the mechanisms powering the Balmer lines in LRDs are distinct. Indeed, the ubiquity of Balmer absorption \citep[e.g.,][]{Lin24} points to resonant scattering \citep[e.g.,][]{Chang25} and collisional excitation \citep[e.g,][]{Torralba25IFU} as opposed to e.g., photo-ionization and dust attenuation that set H$\alpha$ and H$\alpha$/H$\beta$ among classical AGN. **Right:** The Balmer break strength of our stack is among the largest observed with JWST, comparable only to The Cliff \citep[][]{degraaff25} and MoM-BH* \citep[][]{Naidu25BHstar} that are the archetypal BH*s. We highlight that the break strength of the stack lies well above the maximum expected from a standard stellar population assuming a dust-free \citet[][]{Chabrier03} IMF (gray dashed line;  ([Wang and Leja 2024](), [Rix and Naidu 2025]()) ), LRDs with Balmer breaks (red; e.g.,  ([Labbe, Greene and Matthee 2024](), [Wang and Leja 2024](), [Furtak, Labbé and Zitrin 2024](), [Kokorev, Chisholm and Endsley 2024](), [Taylor, Kokorev and Kocevski 2025]()) ), quiescent and mini-quenched galaxies (orange; e.g.,  ([Strait, Brammer and Muzzin 2023](), [Setton and Brammer 2024](), [Weibel and Setton 2024](), [Looser and Maiolino 2024]()) ), and star-forming galaxies (gray points; compiled in  ([Roberts-Borsani, Treu and Shapley 2024]()) ).
     (*fig:context*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2601.20929"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\red}[1]{\textcolor{red}{#1}}$
$\newcommand{\green}[1]{\textcolor{teal}{#1}}$
$\newcommand{\purple}[1]{\textcolor{violet}{#1}}$
$\newcommand{\orcid}[1]$
$\newcommand{\msun}{\ensuremath{M_\odot}\xspace}$
$\newcommand{\Ha}{{\ensuremath{\mathrm{H}\alpha}}\xspace}$
$\newcommand{\reff}{\ensuremath{R_{\rm eff}}\xspace}$</div>



<div id="title">

# $\Euclid$: Early Release Observations $-$ The star-formation history of massive early-type galaxies in the Perseus cluster$\thanks{This paper is published on behalf of the Euclid Consortium.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2601.20948-b31b1b.svg)](https://arxiv.org/abs/2601.20948)<mark>Appeared on: 2026-01-30</mark> -  _21 pages, 8 figures, accepted for publication in A&A_

</div>
<div id="authors">

S. Martocchia, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** The $\Euclid$ Early Release Observations (ERO) programme targeted the Perseus galaxy cluster in its central region over 0.7 deg $^2$ . We combined the exceptional image quality and depth of the ERO-Perseus with FUV and NUV observations from GALEX and AstroSat/UVIT, as well as $ugriz\Ha$ data from MegaCam at the CFHT, to deliver FUV-to-NIR magnitudes of the 87 brightest galaxies within the Perseus cluster. We reconstructed the star-formation history (SFH) of 59 early-type galaxies (ETGs) within the sample, through the spectral energy distribution (SED) fitting code \texttt{CIGALE} and state-of-the-art stellar population (SP) models to reproduce the galactic UV emission from hot, old, low-mass stars (i.e. the UV upturn). In addition, for the six most massive ETGs in Perseus [ stellar masses $\log_{10}(\Mstellar/\msun) \geq 10.3$ ] , we analysed their spatially resolved SP and SFH through a radial SED fitting.  In agreement with our previous work on Virgo ETGs, we found that (i) the majority of the analysed galaxies needs the presence of an UV upturn component to explain their FUV emission, with average temperatures $\langle T_{\rm UV}\rangle \simeq 33 800$ K; (ii) the ETGs of Perseus have grown their stellar masses quickly, with star-formation timescales $\tau\lesssim 1500$ Myr.  We found that all ETGs in the sample have formed more than about 30 \% of their stellar masses at $z\simeq5$ , up to extreme fractions of $\simeq$ 100 \% . At $z\simeq5$ , the stellar masses of the most massive nearby ETGs, which have present-day stellar masses $\log_{10}(\Mstellar/\msun)\gtrsim 10.8$ , are then found to be comparable to those of the red quiescent galaxies observed by JWST at similar redshifts ( $z>4.6$ ).  This study can be extended to ETGs in the 14 000 deg $^2$ extragalactic sky that will soon be observed by $\Euclid$ , in combination with those from other major upcoming surveys (e.g. _Rubin_ /LSST), and UV observations, to ultimately assess whether the massive ETGs that we observe today represent the progeny of the massive high- $z$ JWST red quiescent galaxies.

</div>

<div id="div_fig1">

<img src="tmp_2601.20948/./figs/FUV_NUV_Mass_integr_virgo_perseus.png" alt="Fig5.1" width="50%"/><img src="tmp_2601.20948/./figs/FUV_VIS_Mass_integr_virgo_perseus.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** ${\rm FUV-NUV}$(*left*) and ${\rm FUV}-\IE$(*right*) colours as a function of stellar mass for the Perseus galaxies studied in this work and for the Virgo galaxies from the GUViCS data \citep{voyer14,boselli14}. Filled green squares (diamonds) indicate Perseus ETGs (unclassified) sources from \cite{meusinger20}. Filled blue diamonds are Perseus LTGs, while the open red circle represents the central galaxy of Perseus, namely NGC 1275. Purple open triangles indicate Virgo LTGs, while Virgo ETGs are represented by purple open stars. M87, the central of Virgo, is also highlighted with an open red square. Black dashed lines are the magnitude limit curves for each corresponding colour for the Perseus observations. Circular apertures are defined in the FUV band starting with a radius equal to the $\reff$ in \IE by \cite{EROPerseusOverview}. Apertures were adjusted when needed to fully encompass the FUV emission, and checked against the $\IE$ images to avoid contamination, as in \cite{voyer14}. We refer to Sect. \ref{subsec:integr} for more details. (*fig:fuv_nuv_integr*)

</div>
<div id="div_fig2">

<img src="tmp_2601.20948/./figs/fuv_nuv_only_SMA.png" alt="Fig6.1" width="50%"/><img src="tmp_2601.20948/./figs/fuv_VIS_only_SMA.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** ${\rm FUV-NUV}$(*left*) and ${\rm FUV}-\IE$(*right*) colours as a function of the semimajor axis in kpc. Empty symbols indicate the values of the central colours calculated when a mask is not applied. The dotted lines connect the central unmasked regions to the successive regions. See Sect. \ref{subsec:rad_analys} for more details. Perseus ETGs are indicated with solid lines, while Virgo ETGs from \cite{martocchia25} are represented by dashed lines and purple colours. The grey dash-dotted vertical line indicates the projected radius within which a disc of dust is observed in the $\IE$ images of NGC 1270, NGC 1277, NGC 1278 and NGC 1281, see Sect. \ref{subsec:dust}. (*fig:rad_UV*)

</div>
<div id="div_fig3">

<img src="tmp_2601.20948/./figs/sed_integrated_results_mask.png" alt="Fig1" width="100%"/>

**Figure 1. -** Stellar mass at $z=0$ as a function of the estimated stellar mass at $z\simeq5$. Data are colour-coded by the fraction of assembled stellar mass by $z\simeq5$. The Perseus integrated sample is connected by vertical dashed blue lines to the results of the radial analysis (cyan stars). Open blue upside-down triangles indicate the radial results on Virgo massive ETGs by \cite{martocchia25}.
The red dotted line represents assembled fractions of 100\%, while the solid, dashed, and dot-dashed blue lines indicate the stellar masses found for red quiescent galaxies with JWST at $z>4.6$ by \cite{carnall23, carnall25}, \cite{degraaff25}, and \cite{barrufet25}. Compilation from \cite{antwi-danso23}. (*fig:integr_results*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2601.20948"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# The Third Option: Color Phase Curves to Characterize the Atmospheres of Temperate Rocky Exoplanets

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2601.20966-b31b1b.svg)](https://arxiv.org/abs/2601.20966)<mark>Appeared on: 2026-01-30</mark> -  _14 pages, 6 figures, submission to The Open Journal of Astrophysics_

</div>
<div id="authors">

D. Deming, et al. -- incl., <mark>L. Kreidberg</mark>

</div>
<div id="abstract">

**Abstract:** Detecting and characterizing the atmospheres of rocky exoplanets has proven to be challenging for JWST.  Transit spectroscopy of the TRAPPIST-1 planets has been impacted by the effects of spots and faculae on the host star.  Secondary eclipses have detected hot rocks, but evidence for atmospheres has been difficult to obtain.  However, there is a third option that we call color phase curves.  This method will apply to synchronously rotating non-transiting planets as well as transiting planets.  A color phase curve uses photometry at a long-IR wavelength near the peak of the planetary thermal emission (e.g., 21 microns) divided by photometry at a shorter wavelength where the star dominates more strongly (e.g., 12 microns).  We avoid wavelengths having potentially strong molecular absorption (e.g., 15 microns) to minimize degeneracies in the color phase curve, and we aim to detect and characterize the planetary atmosphere via its longitudinal heat transfer.  The ratio of two wavelengths observed nearly simultaneously is designed to isolate thermal emission from the planet, discriminate against the star, and largely cancel instrumental systematic effects.  Moreover, we show that invoking mass-radius relations, and using self-consistent physical models, will permit the longitudinal heat transfer to be measured independent of the orbital inclination.  Radial velocity surveys are detecting many new exoplanets, including temperate rocky worlds with Earth-like masses. Most of those planets will not transit, but color phase curves have the potential to detect and characterize their atmospheres. \ \

</div>

<div id="div_fig1">

<img src="tmp_2601.20966/./color_curves.png" alt="Fig1" width="100%"/>

**Figure 1. -** Color phase curves (CPCs) for the sum of planets 'b' and 'd' in the Proxima Centauri system.  The CPC is the ratio of the flux in the 21 $\mu$m to 12.8 $\mu$m JWST/MIRI filters, normalized to the stellar flux, with the median value subtracted from the normalized ratio. The CPCs are the sum of both planets, and are shown here for the 2027 visibility period for JWST (lighter when not visible).  Two cases are shown: using a hot rock model for 'b', and a GCM (see text).  Planet 'd' is modeled as a hot rock in both cases.   (*fig: curves1*)

</div>
<div id="div_fig2">

<img src="tmp_2601.20966/./color_curves2.png" alt="Fig2" width="100%"/>

**Figure 2. -** Color phase curves (CPCs) for planets 'b' and 'd' in the Proxima Centauri system, as in Figure \ref{fig: curves1}, except that the CPC for the 'b' planet is here also plotted separately, in addition to being summed with 'd'. Both planets are represented by the two-column model of \citet{lincowski_2023}(see text for description). (*fig: curves2*)

</div>
<div id="div_fig3">

<img src="tmp_2601.20966/./MRplot_paper.png" alt="Fig5" width="100%"/>

**Figure 5. -** Possible mass and orbital inclination of Proxima Centauri b as a function of the amplitude of the CPC (21- vs. 12.8 $\mu$m), where each curve is constrained by a mass radius relation.  The phase curve amplitudes (X-axis) are based on the toy models from \citet{kreidberg_2016}(see text). We adopt M(R) relations from \citet{otegi_2020} and \citet{muller_2024} in order to show that the result is not critically dependent on the exact form of M(R). Dotted lines show the $\pm1\sigma$ error ranges due to imprecision in M(R). Because the curves for different values of heat redistribution are nearly vertical, an observed CPC amplitude will intersect only a small range of heat redistribution values (see text).  (*fig: MRplot*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2601.20966"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\namecn}[1]{\begin{CJK*}{UTF8}{gbsn}({#1})\end{CJK*}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Thermal emission spectra of the ultra-hot Jupiter WASP-33 b

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2601.21072-b31b1b.svg)](https://arxiv.org/abs/2601.21072)<mark>Appeared on: 2026-01-30</mark> -  _Accepted for publication in Astronomy and Astrophysics(12 pages, 16 figures, 5 tables)_

</div>
<div id="authors">

Q. Zou, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** Observations of exoplanetary atmospheres provide critical insights into their chemical composition, formation and evolution history. Ultra-hot Jupiters serve as excellent targets for atmospheric characterization; studies of these planets may yield key understanding of gas giant's formation and evolution history. We present a thermal emission study of WASP-33 b's dayside atmosphere, based on two secondary eclipse observations with CFHT/WIRCam in two specific narrow band filters, namely the CO and CH4 $_{\rm on}$ filters, and archival data with HST/WFC3 and Spitzer. Stellar pulsations of the host star induce some quasi-periodic photometric variations, particularly in the CH4 $_{\rm on}$ band, which are modelled and corrected in the high-precision differential light curves. An eclipse depth of $1565.2^{+228.6}_{-237.5}$ ppm  and $914.3^{+56.1}_{-57.0}$ ppm is determined  for the CO and CH4 $_{\rm on}$ bands, respectively. Combined with HST/WFC3 and Spitzer data, our joint retrieval of WASP-33 b's dayside atmosphere reveals a high metallicity ( [ Fe/H ] $= 1.52^{+0.35}_{-0.52}$ ), high C/O ratio (C/O $= 0.78^{+0.03}_{-0.04}$ ), and a thermal inversion layer, suggesting a formation history involving metal-rich gas accretion. We confirm the presence of the molecules H $_{2}$ O, H $^{-}$ and CO, and report a tentative detection of TiO in the dayside atmosphere of WASP-33 b. Future higher precision observations with JWST may provide better understand constraints on the chemical abundances of oxygen and refractory element abundances to better WASP-33 b's formation and evolutionary pathway.

</div>

<div id="div_fig1">

<img src="tmp_2601.21072/./pictures/CO_filter_data.png" alt="Fig3" width="100%"/>

**Figure 3. -** The CO filter reduced full-frame WIRCam image of WSAP-33 taken on 25 October 2015. The target star is marked as the white square and the final chosen reference stars are marked as the grey circles. (*fig:image*)

</div>
<div id="div_fig2">

<img src="tmp_2601.21072/./pictures/Normalized_ref_plot.png" alt="Fig4" width="100%"/>

**Figure 4. -** Normalized light curves of target star and the candidate reference stars.  (*fig:lc*)

</div>
<div id="div_fig3">

<img src="tmp_2601.21072/./pictures/CO_Filter_RMS_beta.png" alt="Fig5" width="100%"/>

**Figure 5. -** Contour maps of the normalized rms$*\beta^2$ distribution are superimposed on the two-dimensional parameter space defined by $D$ and $N_{\rm RSG}$ in the CO filter data. The pixel scale is 0.306 arcsec $\rm pixel^{-1}$. The minimum value of rms$*\beta^2$ is reached with $D=29$ and $N_{\rm RSG}=3$. (*fig:rmsbeta_CO*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2601.21072"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]{\orcidlink{#1}}$
$\newcommand{\pd}{\phantom{1}}$
$\newcommand{\pp}{\phantom{-}}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.55}$</div>



<div id="title">

# $\Euclid$ preparation: Decomposing components of the extragalactic background light using multi-band intensity mapping cross-correlations

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2601.21111-b31b1b.svg)](https://arxiv.org/abs/2601.21111)<mark>Appeared on: 2026-01-30</mark> -  _21 pages, 11 figures_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** The extragalactic background light (EBL) fluctuations in the optical/near-IR encode the cumulative emission of unresolved galaxies (integrated galaxy light; IGL), diffuse intra-halo light (IHL), and high- $z$ sources from the epoch of reionisation (EoR), but they are difficult to disentangle with auto-spectra alone. We aim to decompose the EBL into its principal constituents using multi-band intensity mapping combined with cosmic shear and galaxy clustering. We develop a joint halo-model framework in which IHL follows a mass- and redshift-dependent luminosity scaling, IGL is set by an evolving Schechter luminosity function, and EoR emission is modelled with Pop II/III stellar emissivities and a binned star-formation efficiency. Diffuse Galactic light and shot noise are included, and cosmic shear is modelled using the nonlinear alignment model for intrinsic alignment. Using mock surveys in a flat $\Lambda$ CDM cosmology with ten spectral bands spanning 0.75--5.0 $\si{\micron}$ in the North Ecliptic Pole deep fields over about $100 \deg^2$ with source detections down to AB = 20.5 for masking, and six redshift bins to $z=2.5$ , we fit auto- and cross-power spectra using a Markov chain Monte Carlo method. The combined SPHEREx $\times$ $\Euclid$ analysis recovers all fiducial parameters within $1 \sigma$ and reduces $1 \sigma$ uncertainties on IHL parameters by 10--35 \% relative to SPHEREx EBL-only, while EoR star-formation efficiency parameters improve by 20--35 \% . Cross-correlations reveal a stronger coupling of IHL than IGL to the shear field, enhancing component separation; conversely, the high- $z$ EoR contribution shows negligible correlation with cosmic shear and galaxy clustering, aiding its isolation in the EBL. Relative to the SPHEREx EBL-only case, the inferred IHL fraction as a function of halo mass is significantly tightened over $10^{11}$ – $10^{14} \si{\solarmass}$ , with uncertainties reduced by 5--30 \% , and the resulting star-formation rate density constraints extend to $z\approx11$ , with uncertainty reductions of 22--31 \% . SPHEREx $\times$ $\Euclid$ provides a robust, systematics-aware route to component-resolved EBL measurements and improved constraints on galaxy formation.

</div>

<div id="div_fig1">

<img src="tmp_2601.21111/./kernel.png" alt="Fig6" width="100%"/>

**Figure 6. -** Kernel of different EBL components and cosmic shear signal as functions of redshift. *Left*: Kernels of IHL (*upper*) and IGL (*lower*) signals in EBL (IGL with masking down to AB = 20.5), respectively. *Right*: Kernels of the galaxy clustering (*upper*) and cosmic shear (*lower*), respectively. (*fig:kernel*)

</div>
<div id="div_fig2">

<img src="tmp_2601.21111/./IHLxWL.png" alt="Fig9.1" width="25%"/><img src="tmp_2601.21111/./rhox.png" alt="Fig9.2" width="25%"/><img src="tmp_2601.21111/./EBLxGal.png" alt="Fig9.3" width="25%"/><img src="tmp_2601.21111/./rhoxg.png" alt="Fig9.4" width="25%"/>

**Figure 9. -** Components of the cross angular power spectrum between EBL at 2.2 $\si{\micron}$ and cosmic shear/galaxy clustering at the tomographic redshift bin with $0.4\!<\!z_{\rm ph}\!<\!0.6$ are shown in the left panels, and their correlation coefficient are shown in the right panels. We use the same colour and type of line to present the same component in the left and right panels. *Top*: The blue solid lines denote the cross-correlation between the total components of EBL, and the green, red, purple, and brown lines represent the cross-correlation of IHL-gravitational (IHL$\times$G), IHL-intrinsic (IHL$\times$I), IGL-gravitational (IGL$\times$G), and IGL-intrinsic (IGL$\times$I) signals, respectively. Note that the value of IHL$\times$I and IGL$\times$I are negative. *Bottom*: The solid blue lines represent the cross-correlation of the total EBL with galaxy clustering. The green and red lines are the IHL-galaxy clustering (IHL$\times$GC) and IGL-galaxy clustering (IGL$\times$GC) signals, respectively. (*fig:cross_sp*)

</div>
<div id="div_fig3">

<img src="tmp_2601.21111/./EBL2.2.png" alt="Fig1" width="100%"/>

**Figure 1. -** Components of the EBL anisotropy power spectrum at 2.2 $\si{\micron}$. Here, $D_{\ell}=\ell (\ell+1) C_{\ell}/2\pi$. The blue solid line denotes the total power spectrum with noise, the green, red, and purple dashed lines denote the power spectra of the IHL, IGL, and EoR signals, respectively. DGL is shown as a brown dotted line, for an intensity level comparable to that of the NEP deep field. The shot noise and instrument noise are shown with pink and grey dash-dotted lines, respectively. The orange crosses with error bars represent mock SPHEREx data. (*fig:EBL_spectra*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2601.21111"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\documentname}{\textsl{Article}}$
$\newcommand{\sectionname}{Section}$
$\newcommand{\teff}{T_\mathrm{eff}}$
$\newcommand{\logg}{\log(g)}$
$\newcommand{\vt}{\nu_t}$
$\newcommand{\feh}{[\mathrm{Fe/H}]}$
$\newcommand{\mh}{[\mathrm{M/H}]}$
$\newcommand{\afe}{[\mathrm{\alpha/Fe}]}$
$\newcommand{\mgfe}{\([\mathrm{Mg/Fe}]\)}$
$\newcommand{\cafe}{\([\mathrm{Ca/Fe}]\)}$
$\newcommand{\sife}{\([\mathrm{Si/Fe}]\)}$
$\newcommand{\tife}{\([\mathrm{Ti/Fe}]\)}$</div>



<div id="title">

# Evaluating classifications of extremely metal-poor candidates selected from Gaia XP spectra

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2601.21292-b31b1b.svg)](https://arxiv.org/abs/2601.21292)<mark>Appeared on: 2026-01-30</mark> -  _26 pages, 12 figures. Submitted to AJ_

</div>
<div id="authors">

R. Thai, et al. -- incl., <mark>H.-W. Rix</mark>

</div>
<div id="abstract">

**Abstract:** $\noindent$ Extremely metal-poor stars are intrinsically rare, but emerging methods exist to accurately classify them from all-sky Gaia XP low-resolution spectra.	To assess their overall accuracy for targeting metal-poor stars, we present a high-resolution spectroscopic followup of 75 very metal-poor candidates selected from the catalog by R. Andrae, V. Chandra, and H. W. Rix.	We discover 2 new extremely metal-poor ( $\rm{[Fe / H]}<-3$ ) stars and 20 new very metal-poor ( $\rm{[Fe/H]} < -2$ ) stars.	Abundances of up to 22 elements are derived from 1D local thermodynamic equilibrium analysis and kinematic parameters are derived using Gaia astrometry and spectroscopic radial velocities.	The chemodynamical properties are mostly consistent with expectations for halo stars, but we discover an Mg-enhanced CEMP star ( $\mathrm{[Mg/Fe]} = 0.89$ ) and an Mg-poor star from an accreted ultra-faint dwarf galaxy.	The Gaia XP metallicity estimates are consistent with our $\rm{[Fe/H]}$ measurements down to $\rm{[Fe/H]}\sim -3.0$ , but estimates worsen in highly extincted regions.	We find that 4 other XP-based metallicity catalogs succeed in mitigating contaminants and can also classify metal-poor stars robustly to $\rm{[Fe/H]}\sim -3.0$ .	Our results demonstrate the utility of Gaia XP spectra for identifying the most metal-poor stars across the Galaxy.

</div>

<div id="div_fig1">

<img src="tmp_2601.21292/./kiel_vt.png" alt="Fig1" width="100%"/>

**Figure 1. -** Stellar parameters for all analyzed compared to scaling relations. Top: effective temperature \( $\teff$ \) versus surface gravity \( \log g \) compared to MIST isochrones  (choiMesaIsochronesStellar2016)  of three different metallicities (\( $\feh$ = -2.0, -2.5, -3.0 \)). We generally recover slightly higher or lower \( $\teff$ \) and \( $\logg$ \) values than expected from the isochrones. Bottom: surface gravity \( \log g \) vs microturbulence \( \nu_{t} \) compared to the empirical scaling relations given in \citet[][B05]{barklemHamburgESORprocess2005}, \citet[][M08]{marinoSpectroscopicPhotometricEvidence2008}, and \citet[][K09]{kirbyMultielementAbundanceMeasurements2009}. (*fig:params*)

</div>
<div id="div_fig2">

<img src="tmp_2601.21292/./bias.png" alt="Fig2" width="100%"/>

**Figure 2. -** Dark current is visible in the reduced data. An example master bias image overplotted with the echelle spectrograph order traces on the CCD detector for the FEROS spectrograph. Note the vertical streaks visible in orange. While the bias frame shows only an instantaneous read of the electron noise, the structure of the vertical streaks is consistent with dark current in both shape and magnitude. We find that these manifest in the emission-like features at the Eu II absorption line at 4129 Å  and across subsequent orders in matching positions along the wavelength solution of our reduced spectra. (*fig:dark*)

</div>
<div id="div_fig3">

<img src="tmp_2601.21292/./dynamics_both.png" alt="Fig3" width="100%"/>

**Figure 3. -** Orbital and kinematic properties for our stars. Left: total energy versus angular momentum, colored by apastron distance \( R_{\mathrm{apo}} \). Right: action space, \( (J_{z} - J_{r}) / J_{\mathrm{tot}}\) against angular momentum fraction \( J_{\phi} / J_{\mathrm{tot}} \). Our orbital properties are consistent with expectations for halo VMP stars. CEMP stars, \( r \)-process enhanced stars, and magnesium unusual stars are highlighted per the legend. (*fig:kinematics*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2601.21292"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# BE Lyncis is not a Black Hole Binary:\\Lessons From _Gaia_ and _Hipparcos_ Astrometry

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2601.22071-b31b1b.svg)](https://arxiv.org/abs/2601.22071)<mark>Appeared on: 2026-01-30</mark> -  _6 pages, 1 figure_

</div>
<div id="authors">

P. Nagarajan, et al. -- incl., <mark>K. El-Badry</mark>, <mark>J. Müller-Horn</mark>

</div>
<div id="abstract">

**Abstract:** BE Lyncis (BE Lyn) is a well-studied high-amplitude $\delta$ Scuti variable star (HADS). Recently, \citet{be_lyn_2026} analyzed a 39-year baseline of times of maximum light of BE Lyn, reporting that it is the most eccentric binary known ( $e \approx 0.9989$ ) and hosts the nearest black hole (BH) known to date. We analyze _Hipparcos_ and _Gaia_ astrometry of BE Lyn, predicting what the observed proper motion anomaly (PMA) over the 25 year baseline between the two missions would be were the companion really a $\gtrsim 17.5 M_{\odot}$ BH. We find that the predicted PMA is at least an order of magnitude larger than the observed value of $\approx 1.7 \pm 0.8$ mas yr $^{-1}$ , regardless of the assumed orientation of the orbit. We  predict the expected _Gaia_ DR3 \texttt{RUWE} for different orientations of the putative BH binary, finding that it ranges from $\approx 2.5$ -- $4.0$ , much larger than the reported value of 1.073. The observed value is instead consistent with a low-mass secondary or a single star. We find that BE Lyncis would have received a 7-parameter acceleration solution if it were a BH binary, in contradiction with its absence from the _Gaia_ DR3 non-single star catalogs. Finally, we show that the reported orbit is impossible because the luminous star would overflow its Roche lobe at periastron, irrespective of inclination. We recommend caution in interpreting light-travel time effect (LTTE) models that require very high eccentricities, face-on inclinations, or large companion masses. The observed pulsation timing variations are most likely simply a result of red noise or pulsation phase evolution.

</div>

<div id="div_fig1">

<img src="tmp_2601.22071/./BE_Lyn_Proper_Motions.png" alt="Fig1" width="100%"/>

**Figure 1. -** Predicted instantaneous proper motions for BE Lyn, assuming the best-fit orbital parameters reported by \citet{be_lyn_2026}. We adopt $\Omega = 60^{\circ}$. We show the predicted _Hipparcos_, _Gaia_ DR2, and _Gaia_ DR3 proper motions as well, with the predicted individual epoch measurements plotted with lower opacity. We zoom in on these astrometric measurements in the lower panels. In these panels, we denote the measured _Hipparcos_ proper motions with dashed lines; the shaded regions represent the corresponding uncertainties. Clearly, the observed and predicted _Hipparcos_ proper motions are quite different. We predict that we would observe a proper motion anomaly of $\Delta \mu_{\alpha*} \approx 31$ mas yr$^{-1}$ and  $\Delta \mu_{\delta} \approx 15$ mas yr$^{-1}$ over a 25-year baseline, far greater than the $1$--$2$ mas yr$^{-1}$ differences that are actually observed. (*fig:proper_motions*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2601.22071"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

125  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

18  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

9  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
